## Exercise : Extracting data from OFET-DB to perform ML

Its great that we have data stored in a standardized manner in our database but now we need to extract data from this database in the form of a table containing descriptors [X] and property [y] to perform ML. This notebook will focus on how we can do that. 

First i want you to use the backup file called 20231206_ofetdb_v2_backup11 and add it to your pgadmin as a practice database. We will avoid working with the original database.

In [3]:
# Connect to the database


import psycopg2
import pandas as pd
import numpy as np
import plotly.express as px


#sample connection details
# pgparams = {
#     "host": "127.0.0.1",
#     "database": "ofetdb_testenv",
#     "user": "postgres",
#     "password": "password",
#     "port": "5432",
# }


# Set max number of displayed columns and rows in Jupyter Notebook
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


pgparams = {
    "host": "127.0.0.1",
    "database": "ofetdb_testML",
    "user": "postgres",
    "password": "password",
    "port": "5432",
}

def read_select_query(query):

    with psycopg2.connect(**pgparams) as conn:

        df = pd.read_sql_query(query, conn)

    return df


## Simplest Scenario :

- assume only single polymer scenarios, no blends first (wt_frac = 1)
- assume only single solvent scenarios, no multiple solvents (vol_frac = 1)
- show device substrate information 
- show film deposition information (spin, blade, etc). We will not use the parameters for now
- dont go into detail of solution treatment, substrate pretreat and post process. Just show if treatment was performed
- show hole mobility information

Follow the code blocks below and we will eventually end up with a table containing this information 

1. Prepare a dataframe containing the sample_Id, citation_type and meta information from the experimental table (This one is done as a practice for you)

In [4]:
## Adding all the experiment information 

# SQL query to fetch the required data
query = '''
    SELECT
        s.sample_id,
        e.citation_type,
        e.meta as experiment_meta
    FROM
        SAMPLE s
    JOIN
        EXPERIMENT_INFO e ON s.exp_id = e.exp_id;
'''

# Use the read_select_query function to execute the query
result_df = read_select_query(query)

# Display the resulting DataFrame
#print(result_df)

result_df

OperationalError: connection to server at "127.0.0.1", port 5432 failed: FATAL:  database "ofetdb_testML" does not exist


remember to keep adding to the existing query to add more information to the table. It is going to get complicated and long soon FYI. 

2. Now to this database add the solution concentration information

In [3]:
## only way to join is through ofet_process

query = '''
    SELECT
        s.sample_id,
        e.citation_type,
        e.meta as experiment_meta,
        sol.concentration
    FROM
        SAMPLE s
    JOIN
        EXPERIMENT_INFO e ON s.exp_id = e.exp_id
    LEFT JOIN
        SOLUTION_MAKEUP_SOLVENT sms ON s.process_id = sms.solution_id
    LEFT JOIN
        SOLUTION sol ON sms.solution_id = sol.solution_id ;
'''

# Use the read_select_query function to execute the query
result_df = read_select_query(query)

result_df

/var/folders/6s/8rz0qzlx7dvf696swxc0rjwm0000gn/T/ipykernel_74618/5376181.py:37: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


,sample_id,citation_type,experiment_meta,concentration
0,1,literature,"{'doi': '10.1039/C5TC02579F', 'publication_typ...",4.00000
1,2,literature,"{'doi': '10.1039/C5TC02579F', 'publication_typ...",4.00000
2,3,literature,"{'doi': '10.1039/C5TC02579F', 'publication_typ...",5.00000
3,4,literature,"{'doi': '10.1039/C5TC02579F', 'publication_typ...",5.00000
4,5,literature,"{'doi': '10.1039/C5TC02579F', 'publication_typ...",5.00000
5,6,literature,"{'doi': '10.1039/C5TC02579F', 'publication_typ...",5.00000
6,7,literature,"{'doi': '10.1039/C5TC02579F', 'publication_typ...",5.00000
7,8,literature,"{'doi': '10.1039/C5TC02579F', 'publication_typ...",5.00000
8,9,literature,"{'doi': '10.1039/C5TC02579F', 'publication_typ...",5.00000
9,10,literature,"{'doi': '10.1039/C5TC02579F', 'publication_typ...",5.00000


3. Now to this table add the solvent information but only consider devices made from single solvent vol frac = 1

In [4]:
## only way to join is through ofet_process

query = '''
    SELECT
        s.sample_id,
        e.citation_type,
        e.meta as experiment_meta,
        sol.concentration
    FROM
        SAMPLE s
    JOIN
        EXPERIMENT_INFO e ON s.exp_id = e.exp_id
    JOIN
        SOLUTION_MAKEUP_SOLVENT sms ON s.process_id = sms.solution_id
    JOIN
        SOLUTION sol ON sms.solution_id = sol.solution_id
    WHERE 
        vol_frac = 1;
'''

# Use the read_select_query function to execute the query
result_df = read_select_query(query)

result_df

/var/folders/6s/8rz0qzlx7dvf696swxc0rjwm0000gn/T/ipykernel_74618/5376181.py:37: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


,sample_id,citation_type,experiment_meta,concentration
0,1,literature,"{'doi': '10.1039/C5TC02579F', 'publication_typ...",4.00000
1,2,literature,"{'doi': '10.1039/C5TC02579F', 'publication_typ...",4.00000
2,3,literature,"{'doi': '10.1039/C5TC02579F', 'publication_typ...",5.00000
3,4,literature,"{'doi': '10.1039/C5TC02579F', 'publication_typ...",5.00000
4,5,literature,"{'doi': '10.1039/C5TC02579F', 'publication_typ...",5.00000
5,6,literature,"{'doi': '10.1039/C5TC02579F', 'publication_typ...",5.00000
6,7,literature,"{'doi': '10.1039/C5TC02579F', 'publication_typ...",5.00000
7,8,literature,"{'doi': '10.1039/C5TC02579F', 'publication_typ...",5.00000
8,9,literature,"{'doi': '10.1039/C5TC02579F', 'publication_typ...",5.00000
9,10,literature,"{'doi': '10.1039/C5TC02579F', 'publication_typ...",5.00000


4. Now to this table add polymer information only for devices made from one polymer (no blends) (wt-frac = 1)

In [5]:
## only way to join is through ofet_process

query = '''
    SELECT
        s.sample_id,
        e.citation_type,
        e.meta as experiment_meta,
        sol.concentration
    FROM
        SAMPLE s
    JOIN
        EXPERIMENT_INFO e ON s.exp_id = e.exp_id
    JOIN
        SOLUTION_MAKEUP_SOLVENT sms ON s.process_id = sms.solution_id
    JOIN
        SOLUTION sol ON sms.solution_id = sol.solution_id
    JOIN 
        SOLUTION_MAKEUP_POLYMER smp ON s.process_id = smp.solution_id
    WHERE 
        sms.vol_frac=1 and smp.wt_frac=1;
'''

# Use the read_select_query function to execute the query
result_df = read_select_query(query)

result_df

/var/folders/6s/8rz0qzlx7dvf696swxc0rjwm0000gn/T/ipykernel_74618/5376181.py:37: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


,sample_id,citation_type,experiment_meta,concentration
0,1,literature,"{'doi': '10.1039/C5TC02579F', 'publication_typ...",4.000
1,2,literature,"{'doi': '10.1039/C5TC02579F', 'publication_typ...",4.000
2,3,literature,"{'doi': '10.1039/C5TC02579F', 'publication_typ...",5.000
3,12,literature,"{'doi': '10.1021/acs.chemmater.7b03019', 'publ...",5.000
4,13,literature,"{'doi': '10.1021/acs.chemmater.7b03019', 'publ...",5.000
5,14,literature,"{'doi': '10.1021/acs.chemmater.8b05224', 'publ...",5.000
6,15,literature,"{'doi': '10.1021/acs.chemmater.8b05224', 'publ...",4.000
7,23,literature,"{'doi': '10.1038/srep00754 ', 'publication_typ...",6.500
8,24,literature,"{'doi': '10.1038/srep00754 ', 'publication_typ...",3.000
9,25,literature,"{'doi': '10.1038/srep00754 ', 'publication_typ...",7.000


5. Now to this table add the device substrate information 

In [6]:
query = '''
    SELECT
        s.sample_id,
        e.citation_type,
        e.meta as experiment_meta,
        sol.concentration,
        dev.params
    FROM
        SAMPLE s
    JOIN
        EXPERIMENT_INFO e ON s.exp_id = e.exp_id
    JOIN
        SOLUTION_MAKEUP_SOLVENT sms ON s.process_id = sms.solution_id
    JOIN
        SOLUTION sol ON sms.solution_id = sol.solution_id
    JOIN 
        SOLUTION_MAKEUP_POLYMER smp ON s.process_id = smp.solution_id
    JOIN
        OFET_PROCESS of ON s.process_id =of.process_id
    JOIN
        DEVICE_FABRICATION dev ON dev.device_fab_id = of.device_fab_id
    
    WHERE 
        sms.vol_frac=1 and smp.wt_frac=1;

'''

# Use the read_select_query function to execute the query
result_df = read_select_query(query)

result_df

/var/folders/6s/8rz0qzlx7dvf696swxc0rjwm0000gn/T/ipykernel_74618/5376181.py:37: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


,sample_id,citation_type,experiment_meta,concentration,params
0,1,literature,"{'doi': '10.1039/C5TC02579F', 'publication_typ...",4.000,"{'channel_width': 1500.0, 'gate_material': 'n-..."
1,2,literature,"{'doi': '10.1039/C5TC02579F', 'publication_typ...",4.000,"{'channel_width': 1500.0, 'gate_material': 'n-..."
2,3,literature,"{'doi': '10.1039/C5TC02579F', 'publication_typ...",5.000,"{'channel_width': 1500.0, 'gate_material': 'n-..."
3,12,literature,"{'doi': '10.1021/acs.chemmater.7b03019', 'publ...",5.000,"{'channel_width': 2000.0, 'gate_material': 'n-..."
4,13,literature,"{'doi': '10.1021/acs.chemmater.7b03019', 'publ...",5.000,"{'channel_width': 2000.0, 'gate_material': 'n-..."
5,14,literature,"{'doi': '10.1021/acs.chemmater.8b05224', 'publ...",5.000,"{'channel_width': 2000.0, 'gate_material': 'n-..."
6,15,literature,"{'doi': '10.1021/acs.chemmater.8b05224', 'publ...",4.000,"{'channel_width': 2000.0, 'gate_material': 'n-..."
7,24,literature,"{'doi': '10.1038/srep00754 ', 'publication_typ...",3.000,"{'channel_width': 4000.0, 'gate_material': 'n-..."
8,25,literature,"{'doi': '10.1038/srep00754 ', 'publication_typ...",7.000,"{'channel_width': 4000.0, 'gate_material': 'n-..."
9,26,literature,"{'doi': '10.1038/srep00754 ', 'publication_typ...",6.500,"{'channel_width': 1400.0, 'gate_material': 'n-..."


6. Now to this table add the film deposition type only not the parameters associated with it

In [7]:
query = '''
    SELECT
        s.sample_id,
        e.citation_type,
        e.meta as experiment_meta,
        sol.concentration,
        dev.params,
        fd.deposition_type
    FROM
        SAMPLE s
    JOIN
        EXPERIMENT_INFO e ON s.exp_id = e.exp_id
    JOIN
        SOLUTION_MAKEUP_SOLVENT sms ON s.process_id = sms.solution_id
    JOIN
        SOLUTION sol ON sms.solution_id = sol.solution_id
    JOIN 
        SOLUTION_MAKEUP_POLYMER smp ON s.process_id = smp.solution_id
    JOIN
        OFET_PROCESS of ON s.process_id=of.process_id
      JOIN
        DEVICE_FABRICATION dev ON dev.device_fab_id = of.device_fab_id
    JOIN
        FILM_DEPOSITION fd ON of.process_id = fd.film_deposition_id
    WHERE 
        sms.vol_frac=1 and smp.wt_frac=1;

'''

# Use the read_select_query function to execute the query
result_df = read_select_query(query)

result_df

/var/folders/6s/8rz0qzlx7dvf696swxc0rjwm0000gn/T/ipykernel_74618/5376181.py:37: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


,sample_id,citation_type,experiment_meta,concentration,params,deposition_type
0,1,literature,"{'doi': '10.1039/C5TC02579F', 'publication_typ...",4.00,"{'channel_width': 1500.0, 'gate_material': 'n-...",spin
1,2,literature,"{'doi': '10.1039/C5TC02579F', 'publication_typ...",4.00,"{'channel_width': 1500.0, 'gate_material': 'n-...",spin
2,3,literature,"{'doi': '10.1039/C5TC02579F', 'publication_typ...",5.00,"{'channel_width': 1500.0, 'gate_material': 'n-...",spin
3,12,literature,"{'doi': '10.1021/acs.chemmater.7b03019', 'publ...",5.00,"{'channel_width': 2000.0, 'gate_material': 'n-...",blade
4,13,literature,"{'doi': '10.1021/acs.chemmater.7b03019', 'publ...",5.00,"{'channel_width': 2000.0, 'gate_material': 'n-...",blade
5,14,literature,"{'doi': '10.1021/acs.chemmater.8b05224', 'publ...",5.00,"{'channel_width': 2000.0, 'gate_material': 'n-...",spin
6,15,literature,"{'doi': '10.1021/acs.chemmater.8b05224', 'publ...",4.00,"{'channel_width': 2000.0, 'gate_material': 'n-...",spin
7,24,literature,"{'doi': '10.1038/srep00754 ', 'publication_typ...",3.00,"{'channel_width': 4000.0, 'gate_material': 'n-...",spin
8,25,literature,"{'doi': '10.1038/srep00754 ', 'publication_typ...",7.00,"{'channel_width': 4000.0, 'gate_material': 'n-...",spin
9,26,literature,"{'doi': '10.1038/srep00754 ', 'publication_typ...",6.50,"{'channel_width': 1400.0, 'gate_material': 'n-...",spin


7. Now to this table add the solution treatment, substrate preatreat and post process informaiton 

1 if treatment is done and 0 if no treatment



In [8]:
query = '''
    SELECT
        s.sample_id,
        e.citation_type,
        e.meta as experiment_meta,
        sol.concentration,
        dev.params,
        fd.deposition_type,
    CASE
        WHEN ststep.treatment_type is not NULL then '1'
        ELSE '0'
    END AS SOLUTION_TREATMENT,
    CASE
        WHEN spstep.treatment_type is not NULL then '1'
        ELSE '0'
    END AS SUBSTRATE_PRETREAT,
    CASE
        WHEN ppstep.treatment_type is not NULL then '1'
        ELSE '0'
    END AS POST_PROCESS
    
    FROM
        SAMPLE s
    JOIN
        EXPERIMENT_INFO e ON s.exp_id = e.exp_id
    JOIN
        SOLUTION_MAKEUP_SOLVENT sms ON s.process_id = sms.solution_id
    JOIN
        SOLUTION sol ON sms.solution_id = sol.solution_id
    JOIN 
        SOLUTION_MAKEUP_POLYMER smp ON s.process_id = smp.solution_id
    JOIN
        OFET_PROCESS of ON s.process_id=of.process_id
    JOIN
        DEVICE_FABRICATION dev ON dev.device_fab_id = of.device_fab_id
    JOIN
        FILM_DEPOSITION fd ON of.process_id = fd.film_deposition_id
    LEFT JOIN
        SOLUTION_TREATMENT st ON st.solution_treatment_id = of.solution_treatment_id
    LEFT JOIN
        SOLUTION_TREATMENT_ORDER sto ON sto.solution_treatment_id =st.solution_treatment_id
    LEFT JOIN
        SOLUTION_TREATMENT_STEP ststep ON ststep.solution_treatment_step_id = sto.solution_treatment_step_id
    LEFT JOIN
       SUBSTRATE_PRETREAT sp ON sp.substrate_pretreat_id = of.substrate_pretreat_id
    LEFT JOIN
        SUBSTRATE_PRETREAT_ORDER spo ON spo.substrate_pretreat_id = sp.substrate_pretreat_id
    LEFT JOIN
        SUBSTRATE_PRETREAT_STEP spstep ON spstep.substrate_pretreat_step_id = spo.substrate_pretreat_step_id
    LEFT JOIN
        POSTPROCESS pp ON pp.postprocess_id = of.postprocess_id 
    LEFT JOIN
        POSTPROCESS_ORDER ppo ON ppo.postprocess_id = pp.postprocess_id
    LEFT JOIN
        POSTPROCESS_STEP ppstep ON ppstep.postprocess_step_id = ppo.postprocess_step_id
    WHERE 
        sms.vol_frac=1 and smp.wt_frac=1;
'''

# Use the read_select_query function to execute the query
result_df = read_select_query(query)

result_df

/var/folders/6s/8rz0qzlx7dvf696swxc0rjwm0000gn/T/ipykernel_74618/5376181.py:37: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


,sample_id,citation_type,experiment_meta,concentration,params,deposition_type,solution_treatment,substrate_pretreat,post_process
0,57,literature,"{'doi': '10.1039/c8sm02517g ', 'publication_ty...",18.00,"{'channel_width': 1000.0, 'gate_material': 'n-...",spin,1,1,1
1,137,laboratory,{'lab_notebook_page': 'RV_1.120'},8.00,"{'channel_width': 2000.0, 'gate_material': 'n-...",shear,0,0,1
2,136,laboratory,{'lab_notebook_page': 'RV_1.120'},NaN,"{'channel_width': 2000.0, 'gate_material': 'n-...",shear,0,0,1
3,128,literature,"{'doi': '10.1002/adma.201102786', 'publication...",8.00,"{'channel_width': 10000.0, 'gate_material': 'g...",blade,0,0,1
4,127,literature,"{'doi': '10.1002/adma.201102786', 'publication...",2.00,"{'channel_width': 10000.0, 'gate_material': 'g...",spin,0,1,1
5,126,literature,"{'doi': '10.1002/adma.201102786', 'publication...",2.00,"{'channel_width': 10000.0, 'gate_material': 'g...",blade,0,1,1
6,125,literature,"{'doi': '10.1002/adma.201102786', 'publication...",2.00,"{'channel_width': 10000.0, 'gate_material': 'g...",blade,0,1,1
7,124,literature,"{'doi': '10.1126/sciadv.1600076', 'publication...",2.00,"{'channel_width': 8800.0, 'gate_material': 'n-...",spin,0,1,0
8,123,literature,"{'doi': '10.1021/acsmaterialslett.1c00320', 'p...",2.00,"{'channel_width': 2000.0, 'gate_material': 'n-...",blade,0,1,1
9,122,literature,"{'doi': '10.1021/acsmaterialslett.1c00320', 'p...",10.00,"{'channel_width': 2000.0, 'gate_material': 'n-...",blade,0,1,1


8. Now to this table add the hole mobility information 

only keep devices that have an actual hole mobility value and is not Null or NAN

In [12]:
query = '''
    SELECT
        s.sample_id,
        e.citation_type,
        e.meta as experiment_meta,
        sol.concentration,
        dev.params,
        fd.deposition_type,
        
    CASE
        WHEN ststep.treatment_type is not NULL then '1'
        ELSE '0'
    END AS SOLUTION_TREATMENT,
    CASE
        WHEN spstep.treatment_type is not NULL then '1'
        ELSE '0'
    END AS SUBSTRATE_PRETREAT,
    CASE
        WHEN ppstep.treatment_type is not NULL then '1'
        ELSE '0'
    END AS POST_PROCESS,
    CAST(meas.data-> 'hole_mobility'->>'value' AS float) AS hole_mobility
    
    FROM
        SAMPLE s
    JOIN
        EXPERIMENT_INFO e ON s.exp_id = e.exp_id
    JOIN
        SOLUTION_MAKEUP_SOLVENT sms ON s.process_id = sms.solution_id
    JOIN
        SOLUTION sol ON sms.solution_id = sol.solution_id
    JOIN 
        SOLUTION_MAKEUP_POLYMER smp ON s.process_id = smp.solution_id
    JOIN
        OFET_PROCESS of ON s.process_id=of.process_id
    JOIN
        DEVICE_FABRICATION dev ON dev.device_fab_id = of.device_fab_id
    JOIN
        FILM_DEPOSITION fd ON of.process_id = fd.film_deposition_id
    LEFT JOIN
        SOLUTION_TREATMENT st ON st.solution_treatment_id = of.solution_treatment_id
    LEFT JOIN
        SOLUTION_TREATMENT_ORDER sto ON sto.solution_treatment_id =st.solution_treatment_id
    LEFT JOIN
        SOLUTION_TREATMENT_STEP ststep ON ststep.solution_treatment_step_id = sto.solution_treatment_step_id
    LEFT JOIN
       SUBSTRATE_PRETREAT sp ON sp.substrate_pretreat_id = of.substrate_pretreat_id
    LEFT JOIN
        SUBSTRATE_PRETREAT_ORDER spo ON spo.substrate_pretreat_id = sp.substrate_pretreat_id
    LEFT JOIN
        SUBSTRATE_PRETREAT_STEP spstep ON spstep.substrate_pretreat_step_id = spo.substrate_pretreat_step_id
    LEFT JOIN
        POSTPROCESS pp ON pp.postprocess_id = of.postprocess_id 
    LEFT JOIN
        POSTPROCESS_ORDER ppo ON ppo.postprocess_id = pp.postprocess_id
    LEFT JOIN
        POSTPROCESS_STEP ppstep ON ppstep.postprocess_step_id = ppo.postprocess_step_id
    JOIN
        MEASUREMENT meas ON s.sample_id = meas.sample_id
        
    WHERE 
        sms.vol_frac=1 
        AND smp.wt_frac=1
        AND (meas.data->> 'hole_mobility' IS NOT NULL OR meas.data->> 'hole_mobility' = 'NAN');
'''

# Use the read_select_query function to execute the query
result_df = read_select_query(query)

result_df

/var/folders/6s/8rz0qzlx7dvf696swxc0rjwm0000gn/T/ipykernel_74618/5376181.py:37: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


,sample_id,citation_type,experiment_meta,concentration,params,deposition_type,solution_treatment,substrate_pretreat,post_process,hole_mobility
0,1,literature,"{'doi': '10.1039/C5TC02579F', 'publication_typ...",4.00,"{'channel_width': 1500.0, 'gate_material': 'n-...",spin,0,0,1,0.110000
1,2,literature,"{'doi': '10.1039/C5TC02579F', 'publication_typ...",4.00,"{'channel_width': 1500.0, 'gate_material': 'n-...",spin,0,1,1,0.290000
2,3,literature,"{'doi': '10.1039/C5TC02579F', 'publication_typ...",5.00,"{'channel_width': 1500.0, 'gate_material': 'n-...",spin,0,1,1,0.230000
3,12,literature,"{'doi': '10.1021/acs.chemmater.7b03019', 'publ...",5.00,"{'channel_width': 2000.0, 'gate_material': 'n-...",blade,0,0,1,0.810000
4,13,literature,"{'doi': '10.1021/acs.chemmater.7b03019', 'publ...",5.00,"{'channel_width': 2000.0, 'gate_material': 'n-...",blade,0,0,1,1.530000
5,14,literature,"{'doi': '10.1021/acs.chemmater.8b05224', 'publ...",5.00,"{'channel_width': 2000.0, 'gate_material': 'n-...",spin,0,0,1,0.900000
6,15,literature,"{'doi': '10.1021/acs.chemmater.8b05224', 'publ...",4.00,"{'channel_width': 2000.0, 'gate_material': 'n-...",spin,0,0,1,1.100000
7,24,literature,"{'doi': '10.1038/srep00754 ', 'publication_typ...",3.00,"{'channel_width': 4000.0, 'gate_material': 'n-...",spin,0,1,1,6.850000
8,25,literature,"{'doi': '10.1038/srep00754 ', 'publication_typ...",7.00,"{'channel_width': 4000.0, 'gate_material': 'n-...",spin,0,1,1,7.250000
9,26,literature,"{'doi': '10.1038/srep00754 ', 'publication_typ...",6.50,"{'channel_width': 1400.0, 'gate_material': 'n-...",spin,0,1,1,5.750000


This is a simple version of the dataframe result_df. There are a couple more things we can now do to the dataframe to make it more machine readable. We no longer need to use sql queries. can just work with the dataframe and pandas

9. The code block below will unpack the data stored in device_substrate_parameters and store it as columns (this one is done for you)

In [14]:
 #unpacking the information stored in device_substrate_parameters

import pandas as pd
from pandas import json_normalize


# # Extract the 'device_substrate_parameters' column and normalize it
params_df = json_normalize(result_df['params'])

# # Concatenate the original DataFrame with the new 'params_df'
result_df = pd.concat([result_df, params_df], axis=1)

# # Drop the original 'device_substrate_parameters' column
result_df_norm = result_df.drop('params', axis=1)

# # Display the resulting DataFrame
result_df_norm

,sample_id,citation_type,experiment_meta,concentration,deposition_type,solution_treatment,substrate_pretreat,post_process,hole_mobility,channel_width,gate_material,channel_length,dielectric_material,dielectric_thickness,electrode_configuration,dielectric_material_2,dielectric_thickness_2,channel_width,gate_material,channel_length,dielectric_material,dielectric_thickness,electrode_configuration,dielectric_material_2,dielectric_thickness_2
0,1,literature,"{'doi': '10.1039/C5TC02579F', 'publication_typ...",4.00,spin,0,0,1,0.110000,1500.0,n-doped Si,80.0,SiO2,300.0,BGTC,NaN,NaN,1500.0,n-doped Si,80.0,SiO2,300.0,BGTC,NaN,NaN
1,2,literature,"{'doi': '10.1039/C5TC02579F', 'publication_typ...",4.00,spin,0,1,1,0.290000,1500.0,n-doped Si,80.0,SiO2,300.0,BGTC,NaN,NaN,1500.0,n-doped Si,80.0,SiO2,300.0,BGTC,NaN,NaN
2,3,literature,"{'doi': '10.1039/C5TC02579F', 'publication_typ...",5.00,spin,0,1,1,0.230000,1500.0,n-doped Si,80.0,SiO2,300.0,BGTC,NaN,NaN,1500.0,n-doped Si,80.0,SiO2,300.0,BGTC,NaN,NaN
3,12,literature,"{'doi': '10.1021/acs.chemmater.7b03019', 'publ...",5.00,blade,0,0,1,0.810000,2000.0,n-doped Si,50.0,SiO2,300.0,BGBC,NaN,NaN,2000.0,n-doped Si,50.0,SiO2,300.0,BGBC,NaN,NaN
4,13,literature,"{'doi': '10.1021/acs.chemmater.7b03019', 'publ...",5.00,blade,0,0,1,1.530000,2000.0,n-doped Si,50.0,SiO2,300.0,BGBC,NaN,NaN,2000.0,n-doped Si,50.0,SiO2,300.0,BGBC,NaN,NaN
5,14,literature,"{'doi': '10.1021/acs.chemmater.8b05224', 'publ...",5.00,spin,0,0,1,0.900000,2000.0,n-doped Si,50.0,SiO2,300.0,BGBC,NaN,NaN,2000.0,n-doped Si,50.0,SiO2,300.0,BGBC,NaN,NaN
6,15,literature,"{'doi': '10.1021/acs.chemmater.8b05224', 'publ...",4.00,spin,0,0,1,1.100000,2000.0,n-doped Si,50.0,SiO2,300.0,BGBC,NaN,NaN,2000.0,n-doped Si,50.0,SiO2,300.0,BGBC,NaN,NaN
7,24,literature,"{'doi': '10.1038/srep00754 ', 'publication_typ...",3.00,spin,0,1,1,6.850000,4000.0,n-doped Si,100.0,SiO2,200.0,BGTC,NaN,NaN,4000.0,n-doped Si,100.0,SiO2,200.0,BGTC,NaN,NaN
8,25,literature,"{'doi': '10.1038/srep00754 ', 'publication_typ...",7.00,spin,0,1,1,7.250000,4000.0,n-doped Si,125.0,SiO2,200.0,BGTC,NaN,NaN,4000.0,n-doped Si,125.0,SiO2,200.0,BGTC,NaN,NaN
9,26,literature,"{'doi': '10.1038/srep00754 ', 'publication_typ...",6.50,spin,0,1,1,5.750000,1400.0,n-doped Si,30.0,SiO2,300.0,BGBC,NaN,NaN,1400.0,n-doped Si,30.0,SiO2,300.0,BGBC,NaN,NaN


In [26]:

# Drop unnecessary columns that won't be used for modeling
#columns_to_drop = ['deposition_type', 'citation_type', 'experiment_meta', 'dielectric_material_2', 'dielectric_thickness_2', 'dielectric_material']
columns_to_drop = ['citation_type', 'experiment_meta', 'dielectric_material_2', 'dielectric_thickness_2', 'dielectric_material']

result_df_norm = result_df_norm.drop(columns=columns_to_drop)

# Display the resulting DataFrame
result_df_norm


KeyError: "['citation_type' 'experiment_meta' 'dielectric_material_2'\n 'dielectric_thickness_2' 'dielectric_material'] not found in axis"

We are now going to do something called one hot encoding to convert columns with textual information into numbers (1 and 0)

In [22]:
result_df_norm.columns

Index(['sample_id', 'concentration', 'deposition_type', 'solution_treatment',
       'substrate_pretreat', 'post_process', 'hole_mobility', 'channel_width',
       'gate_material', 'channel_length', 'dielectric_thickness',
       'electrode_configuration', 'channel_width', 'gate_material',
       'channel_length', 'dielectric_thickness', 'electrode_configuration'],
      dtype='object')

In [24]:
# List of columns to one-hot encode
#columns_to_one_hot_encode = ['film_deposition_type', 'gate_material', 'dielectric_material', 'electrode_configuration']
columns_to_one_hot_encode = ['deposition_type', 'gate_material', 'electrode_configuration']

# Perform one-hot encoding
result_df_hot = pd.get_dummies(result_df_norm, columns=columns_to_one_hot_encode)

# Display the resulting DataFrame with one-hot encoding
result_df_hot


,sample_id,concentration,solution_treatment,substrate_pretreat,post_process,hole_mobility,channel_width,channel_length,dielectric_thickness,channel_width,channel_length,dielectric_thickness,deposition_type_blade,deposition_type_dip,deposition_type_drop,deposition_type_shear,deposition_type_spin,gate_material_PET,gate_material_glass,gate_material_n-doped Si,gate_material_PET,gate_material_glass,gate_material_n-doped Si,electrode_configuration_BGBC,electrode_configuration_BGTC,electrode_configuration_TGBC,electrode_configuration_BGBC,electrode_configuration_BGTC,electrode_configuration_TGBC
0,1,4.00,0,0,1,0.110000,1500.0,80.0,300.0,1500.0,80.0,300.0,0,0,0,0,1,0,0,1,0,0,1,0,1,0,0,1,0
1,2,4.00,0,1,1,0.290000,1500.0,80.0,300.0,1500.0,80.0,300.0,0,0,0,0,1,0,0,1,0,0,1,0,1,0,0,1,0
2,3,5.00,0,1,1,0.230000,1500.0,80.0,300.0,1500.0,80.0,300.0,0,0,0,0,1,0,0,1,0,0,1,0,1,0,0,1,0
3,12,5.00,0,0,1,0.810000,2000.0,50.0,300.0,2000.0,50.0,300.0,1,0,0,0,0,0,0,1,0,0,1,1,0,0,1,0,0
4,13,5.00,0,0,1,1.530000,2000.0,50.0,300.0,2000.0,50.0,300.0,1,0,0,0,0,0,0,1,0,0,1,1,0,0,1,0,0
5,14,5.00,0,0,1,0.900000,2000.0,50.0,300.0,2000.0,50.0,300.0,0,0,0,0,1,0,0,1,0,0,1,1,0,0,1,0,0
6,15,4.00,0,0,1,1.100000,2000.0,50.0,300.0,2000.0,50.0,300.0,0,0,0,0,1,0,0,1,0,0,1,1,0,0,1,0,0
7,24,3.00,0,1,1,6.850000,4000.0,100.0,200.0,4000.0,100.0,200.0,0,0,0,0,1,0,0,1,0,0,1,0,1,0,0,1,0
8,25,7.00,0,1,1,7.250000,4000.0,125.0,200.0,4000.0,125.0,200.0,0,0,0,0,1,0,0,1,0,0,1,0,1,0,0,1,0
9,26,6.50,0,1,1,5.750000,1400.0,30.0,300.0,1400.0,30.0,300.0,0,0,0,0,1,0,0,1,0,0,1,1,0,0,1,0,0


Now lets see all the columns in this database 

In [25]:
column_names = result_df_hot.columns
print(column_names)


Index(['sample_id', 'concentration', 'solution_treatment',
       'substrate_pretreat', 'post_process', 'hole_mobility', 'channel_width',
       'channel_length', 'dielectric_thickness', 'channel_width',
       'channel_length', 'dielectric_thickness', 'deposition_type_blade',
       'deposition_type_dip', 'deposition_type_drop', 'deposition_type_shear',
       'deposition_type_spin', 'gate_material_PET', 'gate_material_glass',
       'gate_material_n-doped Si', 'gate_material_PET', 'gate_material_glass',
       'gate_material_n-doped Si', 'electrode_configuration_BGBC',
       'electrode_configuration_BGTC', 'electrode_configuration_TGBC',
       'electrode_configuration_BGBC', 'electrode_configuration_BGTC',
       'electrode_configuration_TGBC'],
      dtype='object')


### Consolidating Descriptors

We are going to consolidate some of the descriptors into one column. 


Coating :

* film_deposition_type_MGC (dip,Dip,blade, inkjet, shear, wire) - value of 1 if any of these columns are true or else 0
* film_deposition_type_spin
* film_deposition_type_drop

Gate Material :

* gate_material_n_doped Si = ('gate_material_n-doped Si', 'gate_material_Si','gate_material_p-doped Si') 

* gate_material_other = ('gate_material_Al', 'gate_material_Au', 'gate_material_PEDOT:PSS', 'gate_material_PET','gate_material_glass')


Dielectric Material :

* dielectric_material_SiO2
* dielectric_material_other = (
        'dielectric_material_6FDA-DABC',
       'dielectric_material_CYTOP', 'dielectric_material_PAN',
       'dielectric_material_PMMA', 'dielectric_material_PTrFE',
       'dielectric_material_PVP', 'dielectric_material_Shellac',
       'dielectric_material_Si3N4')




In [27]:
result_df_hot

,sample_id,concentration,solution_treatment,substrate_pretreat,post_process,hole_mobility,channel_width,channel_length,dielectric_thickness,channel_width,channel_length,dielectric_thickness,deposition_type_blade,deposition_type_dip,deposition_type_drop,deposition_type_shear,deposition_type_spin,gate_material_PET,gate_material_glass,gate_material_n-doped Si,gate_material_PET,gate_material_glass,gate_material_n-doped Si,electrode_configuration_BGBC,electrode_configuration_BGTC,electrode_configuration_TGBC,electrode_configuration_BGBC,electrode_configuration_BGTC,electrode_configuration_TGBC
0,1,4.00,0,0,1,0.110000,1500.0,80.0,300.0,1500.0,80.0,300.0,0,0,0,0,1,0,0,1,0,0,1,0,1,0,0,1,0
1,2,4.00,0,1,1,0.290000,1500.0,80.0,300.0,1500.0,80.0,300.0,0,0,0,0,1,0,0,1,0,0,1,0,1,0,0,1,0
2,3,5.00,0,1,1,0.230000,1500.0,80.0,300.0,1500.0,80.0,300.0,0,0,0,0,1,0,0,1,0,0,1,0,1,0,0,1,0
3,12,5.00,0,0,1,0.810000,2000.0,50.0,300.0,2000.0,50.0,300.0,1,0,0,0,0,0,0,1,0,0,1,1,0,0,1,0,0
4,13,5.00,0,0,1,1.530000,2000.0,50.0,300.0,2000.0,50.0,300.0,1,0,0,0,0,0,0,1,0,0,1,1,0,0,1,0,0
5,14,5.00,0,0,1,0.900000,2000.0,50.0,300.0,2000.0,50.0,300.0,0,0,0,0,1,0,0,1,0,0,1,1,0,0,1,0,0
6,15,4.00,0,0,1,1.100000,2000.0,50.0,300.0,2000.0,50.0,300.0,0,0,0,0,1,0,0,1,0,0,1,1,0,0,1,0,0
7,24,3.00,0,1,1,6.850000,4000.0,100.0,200.0,4000.0,100.0,200.0,0,0,0,0,1,0,0,1,0,0,1,0,1,0,0,1,0
8,25,7.00,0,1,1,7.250000,4000.0,125.0,200.0,4000.0,125.0,200.0,0,0,0,0,1,0,0,1,0,0,1,0,1,0,0,1,0
9,26,6.50,0,1,1,5.750000,1400.0,30.0,300.0,1400.0,30.0,300.0,0,0,0,0,1,0,0,1,0,0,1,1,0,0,1,0,0


In [28]:
## replacing the pubchem_cid with solvent boiling point

# Get unique PubChem CIDs from the 'solvent_pubchem_cid' column
unique_pubchem_cids = result_df['solvent_pubchem_cid'].unique()

# Display the unique PubChem CIDs
print(unique_pubchem_cids)



KeyError: 'solvent_pubchem_cid'

In [ ]:
# Dictionary mapping PubChem CIDs to boiling points
boiling_point_dict = {
    7964: 132,
    6212: 62,
    7239: 180.1,
    6591: 146,
    7809: 138,
    13229: 238,
    13: 213,
    8030: 84,
    1140: 111,
    7501: 145,
    241: 80,
    6344: 40,
    7503: 179
}

# Add a new column "solvent_boiling_point" based on PubChem CIDs
result_df['solvent_boiling_point'] = result_df['solvent_pubchem_cid'].map(boiling_point_dict)


# Drop unnecessary columns that won't be used for modeling
columns_to_drop = ['solvent_pubchem_cid']
result_df = result_df.drop(columns=columns_to_drop)

# Display the resulting DataFrame
result_df


In [ ]:
column_names = result_df.columns
print(column_names)

In [ ]:
result_df_P3HT = result_df[result_df['polymer_common_name'] == 'P3HT']
result_df_DPP_DTT = result_df[result_df['polymer_common_name'] != 'P3HT']


### P3HT Modeling

In [ ]:
result_df_P3HT = result_df_P3HT.drop(columns='polymer_common_name')
result_df_P3HT

In [ ]:
num_rows, num_columns = result_df_P3HT.shape
print(f"Number of rows: {num_rows}")
print(f"Number of columns: {num_columns}")


### DPP-DTT Modeling

In [ ]:
result_df_DPP_DTT = result_df_DPP_DTT.drop(columns='polymer_common_name')
result_df_DPP_DTT

In [ ]:
num_rows, num_columns = result_df_DPP_DTT.shape
print(f"Number of rows: {num_rows}")
print(f"Number of columns: {num_columns}")


In [ ]:
###EXTRACTION PART2
# make each solution processing, substarte pretreat and post processing type into individual column
# have the step number in each type. if done mixing first then 1 second step should have 2